In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lib.imports import *
from lib.video_processor import VideoProcessor
from lib.misc import *

In [ ]:
interpolate_rate=1
eyes_int_rate=1
eye_model_size=512
input_source = 0 # "example_video/example_vid_original.mp4"
form="onnx"


eye_batch_rate=(interpolate_rate//(eyes_int_rate+1))+1
convert_pt_to_onnx(model_path=f'models/eyes_{eye_model_size}_auto.pt', imgsz=eye_model_size, batch=eye_batch_rate, form=form)

processor = VideoProcessor(eye_model_size=eye_model_size, form=form)
out = processor.start(video_path=input_source, interpolate_rate=interpolate_rate, eyes_int_rate=eyes_int_rate, eye_batch_rate=eye_batch_rate, return_instead=False, disable_stats=False);

In [ ]:
# write_video(out, "example_video/flexpressions_cropped.mp4")

## Training

In [ ]:
prepare_video_annotations_for_training(val_ratio=0.1, multiply_train_data=16)

In [ ]:
# if you don't want to track training with comet don't run this cell and set usecomet=False in the next cell
init_comet(api_key="your_api_key")

In [ ]:
for image_size in [64, 128, 256, 512]: # might get stuck before starting 2nd batch, if this happens just restart this cell
    train_model(pretrained_model='models/yolov8n-pose.pt', save_name=f"training_{image_size}_autotrain", imgsz=image_size, epochs=50, batch=64, workspace="comet_workspacename", usecomet=False)